In [21]:
# import dependencies
import time
import numpy as np
from ipywidgets import widgets
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# import MNIST
mnist = input_data.read_data_sets("data/MNIST", one_hot=False)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [17]:
tf.reset_default_graph()

# set parameters
learning_rate = 0.1
dropout_rate = 0.2
is_training = tf.placeholder(tf.bool)

# create the model (a simple convolutional network)
x = tf.placeholder(tf.float32, (None, 784))
y = tf.placeholder(tf.int32, (None,))

x_reshaped = tf.reshape(x, (-1, 28, 28, 1))

conv1 = tf.layers.conv2d(x_reshaped, filters=32, kernel_size=(5, 5), strides=(2, 2), activation=tf.nn.relu,
                         padding="SAME")
conv2 = tf.layers.conv2d(conv1, filters=64, kernel_size=(5, 5), strides=(2, 2), activation=tf.nn.relu,
                         padding="SAME")
conv3 = tf.layers.conv2d(conv2, filters=128, kernel_size=(5, 5), strides=(2, 2), activation=tf.nn.relu,
                         padding="SAME")

flatten = tf.reshape(conv3, (tf.shape(conv3)[0], conv3.shape[1].value * conv3.shape[2].value * 
                                                 conv3.shape[3].value))

dense1 = tf.layers.dense(flatten, 128)

dense2 = tf.layers.dense(dense1, 64)

dropout = tf.layers.dropout(dense2, rate=dropout_rate, training=is_training)

logits = tf.layers.dense(dropout, 10)

cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.cast(y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
tf.reset_default_graph()

# set parameters
learning_rate = 0.1

# create the model (a simple convolutional network)
x = tf.placeholder(tf.float32, (None, 784))
y = tf.placeholder(tf.int32, (None,))

x_reshaped = tf.reshape(x, (-1, 28, 28, 1))

conv1 = tf.layers.conv2d(x_reshaped, filters=32, kernel_size=(5, 5), strides=(2, 2), activation=tf.nn.relu,
                         padding="SAME")
conv2 = tf.layers.conv2d(conv1, filters=64, kernel_size=(5, 5), strides=(2, 2), activation=tf.nn.relu,
                         padding="SAME")
conv3 = tf.layers.conv2d(conv2, filters=128, kernel_size=(5, 5), strides=(2, 2), activation=tf.nn.relu,
                         padding="SAME")

flatten = tf.reshape(conv3, (tf.shape(conv3)[0], conv3.shape[1].value * conv3.shape[2].value * 
                                                 conv3.shape[3].value))

dense1 = tf.layers.dense(flatten, 128)

dense2 = tf.layers.dense(dense1, 64)

dropout = tf.layers.dropout(dense2, rate=dropout_rate, training=is_training)

with tf.variable_scope("means"):
    means = tf.layers.dense(flatten, 10)

with tf.variable_scope("stds"):
    stds = tf.layers.dense(flatten, 10)

dist = tf.distributions.Normal(loc=means, scale=tf.square(stds))
sampling_steps = 50
samples = dist.sample([sampling_steps])

y_index = tf.stack([tf.range(0, tf.shape(y)[0], delta=1), y], axis=1)

targets = tf.map_fn(lambda x: tf.gather_nd(x, y_index), samples)
targets = tf.tile(tf.expand_dims(targets, -1), [1, 1, tf.shape(samples)[-1]])

diff = samples - targets

loss = tf.reduce_logsumexp(diff, axis=-1)
loss = tf.reduce_mean(loss)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

logits = tf.reduce_mean(samples, axis=0)

correct_prediction = tf.equal(tf.cast(tf.argmax(logits, 1), tf.int32), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
# set parameters
num_iterations = 1000

# train

start = time.time()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  # training
  print("Training started.")
    
  for step in range(num_iterations):
        
    batch_xs, batch_ys = mnist.train.next_batch(64)

    sess.run([train_step], feed_dict={
      x: batch_xs,
      y: batch_ys,
      is_training: True
    })

    if step % 100 == 0 and step > 0:
      
      duration = (time.time() - start) / 100
      print("{:.2f} seconds per batch".format(duration))
    
      # test model
      test_accuracy = sess.run(accuracy, feed_dict={
        x: mnist.test.images,
        y: mnist.test.labels,
        is_training: False
      })
      print("Accuracy: %.2f%%" % (test_accuracy * 100))

      start = time.time()
        
  # evaluation
  accuracy = sess.run(accuracy, feed_dict={
    x: mnist.test.images,
    y: mnist.test.labels
  })

print("\nTrained for %d iterations" % num_iterations)
print("Accuracy: %.2f%%" % (accuracy * 100))

Training started.
0.09 seconds per batch
Accuracy: 92.56%
0.10 seconds per batch
Accuracy: 94.62%
0.10 seconds per batch
Accuracy: 95.92%
0.10 seconds per batch
Accuracy: 96.85%
0.12 seconds per batch
Accuracy: 94.28%


KeyboardInterrupt: 